# Projeto Final de Mineração - SEFAZ


## 1) Preparação do ambiente e importação das bibliotecas necessárias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.insert(1, '/content/drive/My Drive/TrabalhoFinal')
import cleaner

import multiprocessing as mp

import numpy as np
import pandas as pd
import matplotlib as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import math
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 2) Importação e análise inicial do dataset

In [ ]:
df = pd.read_csv('/content/drive/My Drive/TrabalhoFinal/ArquivosPrimeiraPagina.csv')
df

,Title,Content,Label
0,"Ato Declaratorio nº 01, de 2013",\n* Publicado no DOE em 01/02/2013\nATO DECLA...,atos declaratorios
1,"Ato Declaratorio nº 02, de 2013",\n* Publicado no DOE em 01/02/2013\nATO DECLA...,atos declaratorios
2,"Ato Declaratório nº 01, de 1998",ATO DECLARATÓRIO Nº 01/1998\n07/01/1998\n* Pub...,atos declaratorios
3,"Ato Declaratório nº 01, de 1999",ATO DECLARATÓRIO Nº 01/1999\n14/01/1999\n* Pub...,atos declaratorios
4,"Ato Declaratório nº 01, de 2000",ATO DECLARATÓRIO Nº 01/2000\n29/03/2000\n* Pub...,atos declaratorios
...,...,...,...
3607,"Resolução nº 1, de 2018",GOVERNO DO ESTADO DO CEARÁ\nSecretaria da Faze...,resolucoes
3608,"Resolução SEFAZ_SETUR nº 01, de 2019",GOVERNO DO \nESTADO DO CEARÁ\nSecretaria da Fa...,resolucoes
3609,"Resolução SEFAZ_SETUR nº 02, de 2019",GOVERNO DO \nESTADO DO CEARÁ\nSecretaria da Fa...,resolucoes
3610,"Resolução SEFAZ_SETUR nº 03, de 2019",GOVERNO DO \nESTADO DO CEARÁ\nSecretaria da Fa...,resolucoes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3612 entries, 0 to 3611
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    3612 non-null   object
 1   Content  3611 non-null   object
 2   Label    3612 non-null   object
dtypes: object(3)
memory usage: 84.8+ KB


In [ ]:
df.groupby('Label')['Title'].count()

Label
atos declaratorios     671
cf e ctn                 2
comunicados             11
decretos              1020
instrucoes            1373
leis                   276
normas                 118
notas                   84
outras normas           47
resolucoes              10
Name: Title, dtype: int64

In [ ]:
TextoInicial = df['Content'][0]


## 3) Pré-processamento

#### 3.1) Aplicação do pré-processamento
Aqui, nós fazemos todos os textos da coluna "Content" do dataset serem minúsculos e removemos as pontuações, símbolos e números deles para facilitar a aplicação em um algoritmo de aprendizagem. Utilizando o método "pool.map", podemos dividir essa tarefa entre os núcleos da CPU para uma execução mais rápida.

In [ ]:
pool = mp.Pool()
df['Content'] = pool.map(cleaner.clearText, df['Content'])

### 3.2) Comparação entre o texto original e o pré-processado

#### Texto original

In [ ]:
TextoInicial

' \n* Publicado no DOE em 01/02/2013\nATO DECLARATÓRIO Nº  01 /2013\nO SECRETÁRIO DA FAZENDA DO ESTADO DO CEARÁ, no uso de suas atribuições \nlegais.\nRESOLVE:\n1. Revogar, a pedido do contribuinte, o Termo de Acordo FDI/PCDM nº 464, de 14 de \nagosto de 2008, celebrado com a empresa TECNO INDÚSTRIA E COMÉRCIO DE \nCOMPUTADORES LTDA, inscrita no CGF sob o  nº 06.358.939-7.\n2. Este Ato Declaratório gera efeito a partir de 1º de fevereiro de 2013\n3. Publique-se. Cumpra-se. Dê-se ciência à interessada.\nSECRETARIO DA FAZENDA DO ESTADO DO CEARÁ, aos 28 de janeiro de 2013\nCarlos Mauro Benevides Filho\n  SECRETÁRIO DA FAZENDA '

#### Texto após pré-processamento

In [ ]:
df['Content'][1]

'publicado doe ato declaratório secretário fazenda estado ceará uso atribuições legais resolve revogar pedido contribuinte termo acordo fdi pcdm outubro celebrado empresa tecno indústria comércio computadores ltda inscrita cgf sob ato declaratório gera efeito partir fevereiro publique-se cumpra-se dê-se ciência interessada secretario fazenda estado ceará janeiro carlos mauro benevides filho secretário fazenda'

### 3.3) Adição da coluna Class com o número das classes
Por fim, para podermos aplicar o dataset no modelo de aprendizagem, precisamos, também, de uma coluna que relaciona o nome da classe com um número. O código abaixo faz exatamente isso.

In [ ]:
dicClassToLabel = {}
dicLabelToClass = {}
nClass = 0

labels = df["Label"].unique()

for i in labels:
    if i not in dicClassToLabel:
        dicClassToLabel[i] = nClass
        dicLabelToClass[nClass] = i
        nClass += 1
      
def toLabel(x):
    return dicClassToLabel[x]

In [ ]:
df["Class"] = df["Label"].map(toLabel)
df

,Title,Content,Label,Class
0,"Ato Declaratorio nº 01, de 2013",publicado doe ato declaratório secretário faze...,atos declaratorios,0
1,"Ato Declaratorio nº 02, de 2013",publicado doe ato declaratório secretário faze...,atos declaratorios,0
2,"Ato Declaratório nº 01, de 1998",ato declaratório publicado doe secretário faze...,atos declaratorios,0
3,"Ato Declaratório nº 01, de 1999",ato declaratório publicado doe secretário faze...,atos declaratorios,0
4,"Ato Declaratório nº 01, de 2000",ato declaratório publicado doe secretário faze...,atos declaratorios,0
...,...,...,...,...
3607,"Resolução nº 1, de 2018",governo estado ceará secretaria fazenda secret...,resolucoes,9
3608,"Resolução SEFAZ_SETUR nº 01, de 2019",governo estado ceará secretaria fazenda secret...,resolucoes,9
3609,"Resolução SEFAZ_SETUR nº 02, de 2019",governo estado ceará secretaria fazenda secret...,resolucoes,9
3610,"Resolução SEFAZ_SETUR nº 03, de 2019",governo estado ceará secretaria fazenda secret...,resolucoes,9


### 3.4) Salvando o dataset modificado

In [ ]:
df.to_csv('/content/drive/My Drive/TrabalhoFinal/ArquivosProcessados.csv')

## 4) Aplicação do dataset no modelo Complement Naive Bayes

### 4.1) Separação do dataset em treino e teste

In [ ]:
treino, teste = train_test_split(df, train_size=0.75)

### 4.2) Vetorização da coluna "Content" utilizando tanto "CountVectorizer" como "TfidfVectorizer" para comparação

In [ ]:
v_1 = CountVectorizer()
X_treino_1 = v_1.fit_transform(treino["Content"])

v_2 = TfidfVectorizer()
X_treino_2 = v_2.fit_transform(treino["Content"])

### 4.3) Aplicação do dataset de treino no modelo

In [ ]:
modelo_1 = ComplementNB().fit(X_treino_1, treino["Class"])

modelo_2 = ComplementNB().fit(X_treino_2, treino["Class"])

### 4.4) Aplicação do dataset de teste no modelo

In [ ]:
X_teste_1 = v_1.transform(teste["Content"])
X_teste_2 = v_2.transform(teste["Content"])

p_1 = modelo_1.predict(X_teste_1)
p_2 = modelo_2.predict(X_teste_2)

### 4.5) Verificação da precisão, da matriz de confusão e da cross validation

In [ ]:
print("Precisão do modelo_1:", ((p_1 == teste["Class"]).sum() / len(p_1)) * 100, "%")
print("Precisão do modelo_2:", ((p_2 == teste["Class"]).sum() / len(p_2)) * 100, "%")

Precisão do modelo_1: 94.46290143964563 %
Precisão do modelo_2: 89.59025470653378 %


In [ ]:
print(metrics.confusion_matrix(teste["Class"], p_1))
print(metrics.confusion_matrix(teste["Class"], p_2))

[[171   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0]
 [  1   0   0 230   5   2   0   0   0   0]
 [  1   0   0  10 346   0   0   0   0   0]
 [  0   0   0   5   2  62   0   0   1   0]
 [  0   0   0   0   6   0  24   0   1   0]
 [  1   0   0   4   0   1   0  15   0   0]
 [  0   0   0   4   0   4   0   0   3   0]
 [  0   0   0   0   0   0   0   0   0   2]]
[[171   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0]
 [  1   0   0   0   0   0   0   0   0   0]
 [  1   0   0 223  14   0   0   0   0   0]
 [  1   0   0   9 347   0   0   0   0   0]
 [  0   0   0  18   5  46   0   0   1   0]
 [  0   0   0   3  13   0  14   0   1   0]
 [  5   0   0  10   1   0   0   5   0   0]
 [  0   0   0   7   1   2   0   0   1   0]
 [  0   0   0   0   0   0   0   0   0   2]]


In [ ]:
cNB = ComplementNB()
scores_1 = cross_val_score(cNB, CountVectorizer().fit_transform(treino["Content"]), treino["Class"], cv=5)
scores_2 = cross_val_score(cNB, TfidfVectorizer().fit_transform(treino["Content"]), treino["Class"], cv=5)

print("Média da Cross Validation com CountVectorizer:", scores_1.mean() * 100, "%")
print("Média da Cross Validation com TfidfVectorizer:", scores_2.mean() * 100, "%")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Média da Cross Validation com CountVectorizer: 93.76151857636876 %
Média da Cross Validation com TfidfVectorizer: 89.55310310958933 %


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


## 5) Aplicação do dataset de treino em uma cross validation com o modelo de Regressão Logística


In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
scores_1 = cross_val_score(lr, CountVectorizer().fit_transform(treino["Content"]), treino["Class"], cv=5)
scores_2 = cross_val_score(lr, TfidfVectorizer().fit_transform(treino["Content"]), treino["Class"], cv=5)

print("Média da Cross Validation com CountVectorizer:", scores_1.mean() * 100, "%")
print("Média da Cross Validation com TfidfVectorizer:", scores_2.mean() * 100, "%")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown 

Média da Cross Validation com CountVectorizer: 97.5639617764015 %
Média da Cross Validation com TfidfVectorizer: 93.65054463853326 %
